## Collection
Gathers all essays from the [UOL site](https://educacao.uol.com.br/bancoderedacoes/).


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get('https://educacao.uol.com.br/bancoderedacoes/')

In [4]:
button_more = True
thumbnails = []
while button_more:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(2)
    try:
        driver.find_element(By.CLASS_NAME, 'ver-mais').click()
        sleep(1)
    except:
        button_more = False
sleep(2)


In [5]:
content = driver.page_source
soup = BeautifulSoup(content)
thumbnails.append(soup.findAll('div', attrs={'class':'thumbnails-wrapper'}))

In [6]:
flat_list = [item for sublist in thumbnails for item in sublist]
len(flat_list)

55

In [7]:
links = []
for thumbnail in flat_list:
    links.append(thumbnail.find('a')['href'])

In [8]:
links = list(dict.fromkeys(links))
links

['https://educacao.uol.com.br/bancoderedacoes/propostas/qualificacao-e-o-futuro-do-emprego.htm',
 'https://educacao.uol.com.br/bancoderedacoes/propostas/supremo-tribunal-federal-e-opiniao-publica.htm',
 'https://educacao.uol.com.br/bancoderedacoes/propostas/ciencia-tecnologia-e-superacao-dos-limites-humanos.htm',
 'https://educacao.uol.com.br/bancoderedacoes/propostas/um-reu-deve-ou-nao-ser-preso-apos-a-condenacao-em-2-instancia.htm',
 'https://educacao.uol.com.br/bancoderedacoes/propostas/agrotoxicos-ou-defensivos-agricolas-dois-nomes-e-uma-polemica.htm',
 'https://educacao.uol.com.br/bancoderedacoes/propostas/preservacao-da-amazonia-desafio-brasileiro-ou-internacional.htm',
 'https://educacao.uol.com.br/bancoderedacoes/propostas/criptomoeda-tecnologia-e-revolucao-economica.htm',
 'https://educacao.uol.com.br/bancoderedacoes/propostas/a-ciencia-na-era-da-pos-verdade.htm',
 'https://educacao.uol.com.br/bancoderedacoes/propostas/universidade-em-crise-quem-paga-a-conta.htm',
 'https://ed

In [9]:
other_format_index = links.index('https://educacao.uol.com.br/bancoderedacoes/propostas/violencia-e-drogas-o-papel-do-usuario.htm')
links = links[0:other_format_index]
links

['https://educacao.uol.com.br/bancoderedacoes/propostas/qualificacao-e-o-futuro-do-emprego.htm',
 'https://educacao.uol.com.br/bancoderedacoes/propostas/supremo-tribunal-federal-e-opiniao-publica.htm',
 'https://educacao.uol.com.br/bancoderedacoes/propostas/ciencia-tecnologia-e-superacao-dos-limites-humanos.htm',
 'https://educacao.uol.com.br/bancoderedacoes/propostas/um-reu-deve-ou-nao-ser-preso-apos-a-condenacao-em-2-instancia.htm',
 'https://educacao.uol.com.br/bancoderedacoes/propostas/agrotoxicos-ou-defensivos-agricolas-dois-nomes-e-uma-polemica.htm',
 'https://educacao.uol.com.br/bancoderedacoes/propostas/preservacao-da-amazonia-desafio-brasileiro-ou-internacional.htm',
 'https://educacao.uol.com.br/bancoderedacoes/propostas/criptomoeda-tecnologia-e-revolucao-economica.htm',
 'https://educacao.uol.com.br/bancoderedacoes/propostas/a-ciencia-na-era-da-pos-verdade.htm',
 'https://educacao.uol.com.br/bancoderedacoes/propostas/universidade-em-crise-quem-paga-a-conta.htm',
 'https://ed

In [11]:
import re
import copy
import os

for link, prompt_number in zip(links, range(len(links))):
    driver.get(link)
    sleep(2)
    content = driver.page_source
    soup = BeautifulSoup(content)
    
    #Title and essay prompt
    title = soup.find('i', attrs={'class':'custom-title'}).text.strip()
    prompt = soup.find('div', attrs={'class':'text'}).text.strip()
    
    os.makedirs('prompts/prompt'+str(prompt_number)+'/essays/', exist_ok=True)
    with open('prompts/prompt'+str(prompt_number)+'/prompt.txt', 'w') as file:
        file.write(title + "\n\n")
        file.write(prompt)

    # Gets all essays
    list_of_links = soup.find('article', attrs={'class':'rt-body'}).findAll('a')
    links_to_essays = []
    for a in list_of_links:
        links_to_essays.append(a['href'])

    for essay, essay_number in zip(links_to_essays, range(len(links_to_essays))):
        driver.get(essay)
        content = driver.page_source
        soup = BeautifulSoup(content)
        composition = soup.find('div', attrs={'class':'container-composition'})
        title = composition.find('h2').text
        original = copy.copy(composition.find('div', attrs={'class':'text-composition'}))
        for elm in original.find_all('span', style=re.compile(r"color:#00b050")):
            elm.string = ""

        corrected = copy.copy(composition.find('div', attrs={'class':'text-composition'}))
        for elm in corrected.find_all('strong'):
            elm.string = ""
        
        results_table = soup.find("section", attrs={'class':'results-table'})
        topics = [topic.text for topic in results_table.findAll('span', attrs={'class':'topic'})]
        points = [int(points.text) for points in results_table.findAll('span', attrs={'class':'points'})]
        df=pd.DataFrame({'Topics':topics, 'Points': points})
        
        with open('prompts/prompt'+str(prompt_number)+'/essays/essay'+str(essay_number)+'_original.txt', 'w', encoding='utf-8') as file:
            file.write(title + "\n\n")
            file.write(original.text + "\n\n")
        
        with open('prompts/prompt'+str(prompt_number)+'/essays/essay'+str(essay_number)+'_corrected.txt', 'w', encoding='utf-8') as file:
            file.write(title + "\n\n")
            file.write(corrected.text + "\n\n")
        df.to_csv('prompts/prompt'+str(prompt_number)+'/essays/essay'+str(essay_number)+'_mark.csv', index=False)
            

AttributeError: 'NoneType' object has no attribute 'find'